In [1]:
!pip install datasets transformers evaluate

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     -------------------------------------- 452.9/452.9 kB 2.2 MB/s eta 0:00:00
     ---------------------------------------- 5.8/5.8 MB 12.9 MB/s eta 0:00:00
     ---------------------------------------- 81.4/81.4 kB ? eta 0:00:00
     ---------------------------------------- 182.4/182.4 kB ? eta 0:00:00
     -------------------------------------- 132.9/132.9 kB 8.2 MB/s eta 0:00:00
     --------------------------------------- 10.9/10.9 MB 15.2 MB/s eta 0:00:00
     ---------------------------------------- 139.5/139.5 kB ? eta 0:00:00
     --------------------------------------- 20.3/20.3 MB 14.9 MB/s eta 0:00:00
     ---------------------------------------- 110.5/110.5 kB ? eta 0:00:00
     ------------------------------------- 267.8/267.8 kB 17.2 MB/s eta 0:00:00
     ---------------------------------------- 3.3/3.3 MB 17.6 MB/s eta 0:00:00


In [6]:
from datasets import load_dataset

In [7]:
dataset = load_dataset("yelp_review_full")

Generating train split:   0%|          | 0/650000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset yelp_review_full downloaded and prepared to /root/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})

In [11]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)
tokenized_datasets = dataset.map(tokenize_function, batched=True)

  0%|          | 0/650 [00:00<?, ?ba/s]

  0%|          | 0/50 [00:00<?, ?ba/s]

In [14]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [15]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [16]:
# from transformers import TrainingArguments

# training_args = TrainingArguments(output_dir="test_trainer")

In [20]:
# !pip install evaluate
import evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 81 kB 8.0 MB/s 


In [25]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch", num_train_epochs=10)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [26]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1000
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1250
  Number of trainable parameters = 108314117


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.082820,0.542000
2,No log,1.198731,0.549000
3,No log,1.485908,0.536000
4,0.640600,1.584748,0.554000
5,0.640600,1.968717,0.578000
6,0.640600,2.462809,0.557000
7,0.640600,2.697876,0.568000
8,0.087100,2.806592,0.568000


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
Saving model che

KeyboardInterrupt: ignored

## `huggingface`

In [12]:
import subprocess
import psycopg2
import pandas as pd

In [14]:

creds = {'dbname': 'd1mikus7g5uss8',
 'host': 'ec2-99-81-68-240.eu-west-1.compute.amazonaws.com',
 'port': '5432',
 'user': 'iohznziolcottb',
 'password': '5a812ea29f6142328bc2afab03e48e6462939babe87610342cdf12e2d357a4f0',
 'sslmode': 'require'}

creds

{'dbname': 'd1mikus7g5uss8',
 'host': 'ec2-99-81-68-240.eu-west-1.compute.amazonaws.com',
 'port': '5432',
 'user': 'iohznziolcottb',
 'password': '5a812ea29f6142328bc2afab03e48e6462939babe87610342cdf12e2d357a4f0',
 'sslmode': 'require'}

In [15]:
conn = psycopg2.connect(host=creds['host'], database=creds['dbname'],
                                user=creds['user'],
                                password=creds['password'])
cur = conn.cursor()

In [16]:
select_Query = 'select * from public."DiscH_prototype_question"'
cur.execute(select_Query)
questions = cur.fetchall()
question_col = ["question_id", "description", "num_response", "label_in_question"]
questions_df = pd.DataFrame(questions, columns=question_col)

select_Query = 'select * from public."DiscH_prototype_answer"'
cur.execute(select_Query)
answers = cur.fetchall()
answer_col = ["answer_id", "answer_category_num", "answer_justification", "answer_upvote",
              "account_id_id", "question_id_id", "date"]
answers_df = pd.DataFrame(answers, columns=answer_col)

select_Query = 'select * from public."DiscH_prototype_answer_bow"'
cur.execute(select_Query)
answers_bow = cur.fetchall()
answer_bow_col = ["answer_id", "answer_category_num", "answer_justification", "answer_upvote",
              "account_id_id", "question_id_id", "answer_text_comment", "date"]
answers_bow_df = pd.DataFrame(answers_bow, columns=answer_bow_col)


select_Query = 'select * from public."DiscH_prototype_achievement"'
cur.execute(select_Query)
achievements = cur.fetchall()
achievement_col = ["achievement_id", "achievement_type", "achievement_date", "account_id", "question_id_id", "value"]
achievements_df = pd.DataFrame(achievements, columns=achievement_col)

In [17]:
select_Query = 'select q.description, a.answer_category_num, a.answer_justification, a.answer_upvote, \
              a.account_id_id from public."DiscH_prototype_question" q RIGHT JOIN public."DiscH_prototype_answer" a \
ON a.question_id_id=q.question_id'
cur.execute(select_Query)
QAs = cur.fetchall() 
QA_col = ["question_description", "answer_category_num", "answer_justification", "answer_upvote", "account_id"]
QA_df = pd.DataFrame(QAs, columns=QA_col)

In [20]:
clean_data = QA_df.copy()
clean_data  = clean_data[['question_description', 'answer_category_num']]

In [22]:
clean_data.drop_duplicates(subset='question_description', inplace=True)
clean_data.reset_index(drop=True, inplace=True)

In [25]:
# checking how many comments have different votes
for i, row in clean_data.iterrows():

    comment = row.question_description

    # getting the dataframe for that comment
    temp_df = clean_data[clean_data.question_description == comment]
    
    # how many unique answers that comment has
    n_ = temp_df.answer_category_num.nunique()

    # changing the answer_category_num to the most frequent one
    if n_ >= 2:
        most_voted_label = temp_df.answer_category_num.value_counts().index[0]
        clean_data.at[i, 'answer_category_num'] = most_voted_label

In [28]:
clean_data

,question_description,answer_category_num
0,وزير الخارجية اللبناني جبران باسيل قال في سلسل...,Religious affiliation
1,سورية بلد الحضارات تربطها بعلية او بحيوان,Violent
2,تقتلون وسام الحسن وتترحموعلية من أي أصناف المخ...,Racist
3,معك خبر انو بلدة قطر متل ما سميتا مساحتها اكبر...,Normal
4,للامانه قوت الموسم اللي طاف كان هوا بس متحمس ح...,Normal
...,...,...
2250,لقاء خلدة قادر ان يسمي شيخ فتنة او شيخ تحريض ا...,Religious affiliation
2251,اكثر ثلاث كلمات متداولة من الاكثر للاقل باسيل ...,Mockery
2252,اجواء اوروبيه بواقع صومالي زنجي حقير,Racist
2253,قداي بدك نمرة الضبان خبرني حتى شوف اذا في قياس...,Violent


In [47]:
from datasets import Dataset

dataset = Dataset.from_pandas(clean_data).train_test_split(test_size=0.15)

In [48]:
dataset

DatasetDict({
    train: Dataset({
        features: ['question_description', 'answer_category_num'],
        num_rows: 1916
    })
    test: Dataset({
        features: ['question_description', 'answer_category_num'],
        num_rows: 339
    })
})

In [49]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
def tokenize_function(examples):
    return tokenizer(examples["question_description"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-cased/snapshots/5532cc56f74641d4bb33641f5c76a55d11f846e0/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--bert-base-cased/snapshots/5532cc56f74641d4bb33641f5c76a55d11f846e0/vocab.txt
loading file tokenize

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [61]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=6)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-cased/snapshots/5532cc56f74641d4bb33641f5c76a55d11f846e0/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transfo

In [62]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch", num_train_epochs=10)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [63]:
train_dataset = tokenized_datasets["train"] 
eval_dataset = tokenized_datasets["test"] 

In [64]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

OutOfMemoryError: ignored

## `pytorch LSTM model`

In [2]:
!pip install torchtext

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ---------------------------------------- 1.9/1.9 MB 5.4 MB/s eta 0:00:00
     ------------------------------------- 162.5/162.5 MB 23.4 MB/s eta 0:00:00


In [81]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import time

import torch
from torchtext import data
import torch.nn as nn

In [82]:
!pip install torchtext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [78]:
data.

,question_description,answer_category_num
0,وزير الخارجية اللبناني جبران باسيل قال في سلسل...,Religious affiliation
1,سورية بلد الحضارات تربطها بعلية او بحيوان,Violent
2,تقتلون وسام الحسن وتترحموعلية من أي أصناف المخ...,Racist
3,معك خبر انو بلدة قطر متل ما سميتا مساحتها اكبر...,Normal
4,للامانه قوت الموسم اللي طاف كان هوا بس متحمس ح...,Normal
...,...,...
2250,لقاء خلدة قادر ان يسمي شيخ فتنة او شيخ تحريض ا...,Religious affiliation
2251,اكثر ثلاث كلمات متداولة من الاكثر للاقل باسيل ...,Mockery
2252,اجواء اوروبيه بواقع صومالي زنجي حقير,Racist
2253,قداي بدك نمرة الضبان خبرني حتى شوف اذا في قياس...,Violent


In [69]:
X, y = clean_data[["question_description"]], clean_data["answer_category_num"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)

X_train.shape, X_test.shape

((2029, 1), (226, 1))

In [ ]:
train_df, valid_df = train_test_split(train)

In [70]:
# reproducible results
SEED = 42

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [72]:
# TEXT = data.Field(tokenize = 'spacy', include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)

AttributeError: ignored

In [ ]:
data.